In [30]:
!pip install cassandra-driver

In [31]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

cloud_config = {
    "secure_connect_bundle": "secure-connect-nyc-yellow-taxi-db.zip"  # replace <</PATH/TO/>> with the path where your downloaded bundle was downloaded (make sure to place this python file in the same place as well)
}
auth_provider = PlainTextAuthProvider(
    "yiGvbsUpoEqllXBsXXwOjZBc",
    "p,b-6bj-cMT1IR3dv,kGm6A8Q+yZg0jcMHWivO2XU0y9+evSLpYZ923mLvs,ocJ-CLeOdYI8EeiBoa7jR.r3fOFtpiQwO+Zdb2aq4FqaO4xSEXj_TF4KFkHoAs5N6mt_",
)  # replace <<CLIENT ID>> and <<CLIENT SECRET>> with the ClientID and Client Secret from your generated token
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()
session.set_keyspace(
    "nyc_yellow_taxi_db")  # replace <<KEYSPACENAME>> with the name of the keyspace you created in DataStax

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(138459394081216) c072af12-8ab0-4392-856a-67d9d9a3c4f5-us-east1.db.astra.datastax.com:29042:b52f7bbc-23dd-487d-bcb5-579b3e9c330a> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


In [33]:
# -✔️- Reading listings from CSV into dataframe

import pandas as pd

# read the csv files into the dataframes
trip_data = pd.read_csv('taxi_trip_data.csv')
zone_data = pd.read_csv('taxi_zone_geo.csv')

In [34]:
# a) Remove the columns “store_and_fwd_flag”, “rate_code” and “total_amount” from taxitripdata
trip_data = trip_data.drop(['store_and_fwd_flag', 'rate_code', 'total_amount'], axis=1)


In [35]:
# b) Drop rows with missing details
trip_data = trip_data.dropna()


In [36]:
trip_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204499 entries, 0 to 204498
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   vendor_id            204499 non-null  int64  
 1   pickup_datetime      204499 non-null  object 
 2   dropoff_datetime     204499 non-null  object 
 3   passenger_count      204499 non-null  float64
 4   trip_distance        204499 non-null  float64
 5   payment_type         204499 non-null  float64
 6   fare_amount          204499 non-null  float64
 7   extra                204499 non-null  float64
 8   mta_tax              204499 non-null  float64
 9   tip_amount           204499 non-null  float64
 10  tolls_amount         204499 non-null  float64
 11  imp_surcharge        204499 non-null  float64
 12  pickup_location_id   204499 non-null  float64
 13  dropoff_location_id  204499 non-null  float64
dtypes: float64(11), int64(1), object(2)
memory usage: 23.4+ MB


In [37]:
zone_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   zone_id    263 non-null    int64 
 1   zone_name  263 non-null    object
 2   borough    263 non-null    object
 3   zone_geom  263 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.3+ KB


In [ ]:
# columns in the trip_data table:
# vendor_id, pickup_datetime, dropoff_datetime, passenger_count, trip_distance, rate_code, store_and_fwd_flag, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount,
# imp_surcharge, total_amount, pickup_location_id, dropoff_location_id, pickup_location, dropoff_location
#  zone_id_x zone_name_x  borough_x  zone_geom_x  zone_id_y zone_name_y  borough_y  zone_geom_y

# session.execute(
#     """
#     DROP TABLE trip_data;
#     """
# )

# create the table for trip_data
session.execute(
    """
CREATE TABLE trip_data (
    trip_id UUID PRIMARY KEY,
    vendor_id INT,
    pickup_datetime TIMESTAMP,
    dropoff_datetime TIMESTAMP,
    passenger_count FLOAT,
    trip_distance FLOAT,
    payment_type INT,
    fare_amount FLOAT,
    extra FLOAT,
    mta_tax FLOAT,
    tip_amount FLOAT,
    tolls_amount FLOAT,
    imp_surcharge FLOAT,
    pickup_location INT,
    dropoff_location INT,
);
    """
)

In [ ]:
# c) Insert the data in the database as you see fit

# Insert the data into the trip_data table
# instert only 20,000 rows

for index, row in trip_data.head(20000).iterrows():

    # Extract data for insertion
    vendor_id = row['vendor_id']
    pickup_datetime = row['pickup_datetime']
    dropoff_datetime = row['dropoff_datetime']
    passenger_count = row['passenger_count']
    trip_distance = row['trip_distance']
    payment_type = row['payment_type']
    fare_amount = row['fare_amount']
    extra = row['extra']
    mta_tax = row['mta_tax']
    tip_amount = row['tip_amount']
    tolls_amount = row['tolls_amount']
    imp_surcharge = row['imp_surcharge']
    pickup_location = row['pickup_location_id']
    dropoff_location = row['dropoff_location_id']

    # Insert the data into the trip_data table
    session.execute(
        """
        INSERT INTO trip_data (trip_id,vendor_id, pickup_datetime, dropoff_datetime, passenger_count, trip_distance, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, imp_surcharge, pickup_location, dropoff_location)
        VALUES (UUID(), %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
        (
            vendor_id,
            pickup_datetime,
            dropoff_datetime,
            passenger_count,
            trip_distance,
            payment_type,
            fare_amount,
            extra,
            mta_tax,
            tip_amount,
            tolls_amount,
            imp_surcharge,
            pickup_location,
            dropoff_location,
        ),
    )

In [27]:
# session.execute("ALTER TABLE trip_data ADD duration float;")
session.execute("ALTER TABLE trip_data ADD total_trip_cost float;")


In [24]:
# d) Calculate the duration for each trip and add it as a new field in your database (e7sb el duration men pickup date le dropoff date)
# add a new column to the trip_data dataframe for the duration of the trip
# Query for trips
rows = session.execute("SELECT trip_id, pickup_datetime, dropoff_datetime FROM trip_data")

# Update each row with duration_seconds
for row in rows:
    pickup_datetime = row.pickup_datetime
    dropoff_datetime = row.dropoff_datetime

    # Calculate duration in seconds
    duration = (dropoff_datetime - pickup_datetime).total_seconds()

    # Update the row with duration_seconds
    session.execute(
        """
        UPDATE trip_data
        SET duration = %s
        WHERE trip_id = %s
        """,
        (duration, row.trip_id)
    )

In [29]:
# e) Use “fare_amount”, “extra”, “mta_tax”, “tip_amount”, “tolls_amount” and “imp_surcharge” to calculate the total trip cost and add it as a new field in your database (e7sb el total cost)

rows = session.execute("SELECT trip_id, fare_amount, extra, mta_tax, tip_amount, tolls_amount, imp_surcharge FROM trip_data")

# Update each row with total_trip_cost
for row in rows:
    fare_amount = row.fare_amount
    extra = row.extra
    mta_tax = row.mta_tax
    tip_amount = row.tip_amount
    tolls_amount = row.tolls_amount
    imp_surcharge = row.imp_surcharge

    # Calculate total trip cost
    total_trip_cost = fare_amount + extra + mta_tax + tip_amount + tolls_amount + imp_surcharge

    # Update the row with total_trip_cost
    session.execute(
        """
        UPDATE trip_data
        SET total_trip_cost = %s
        WHERE trip_id = %s
        """,
        (float(total_trip_cost), row.trip_id)
         )


In [39]:
# f) What is the most common payment type used per time of day? Hint: time of day meaning morning, afternoon, evening, night
# create a new column in the trip_data dataframe for the time of day
# Create a new column to store the time of day
session.execute("ALTER TABLE trip_data ADD time_of_day TEXT")

In [41]:
# Update each row with the time of day
rows = session.execute("SELECT trip_id, pickup_datetime FROM trip_data")
for row in rows:
    pickup_hour = row.pickup_datetime.hour
    if pickup_hour < 6:
        time_of_day = 'Night'
    elif pickup_hour < 12:
        time_of_day = 'Morning'
    elif pickup_hour < 18:
        time_of_day = 'Afternoon'
    else:
        time_of_day = 'Evening'

    # Update the row with the time of day
    session.execute(
        """
        UPDATE trip_data
        SET time_of_day = %s
        WHERE trip_id = %s
        """,
        (time_of_day, row.trip_id)
    )

In [48]:
# Fetch all rows from Cassandra
rows = session.execute("SELECT pickup_datetime, payment_type FROM trip_data")

# Initialize empty dictionary to store payment counts for each time of day
payment_counts = {}

# Iterate over rows and update payment counts for each time of day
for row in rows:
    pickup_datetime = row.pickup_datetime
    pickup_hour = pickup_datetime.hour
    if pickup_hour < 6:
        time_of_day = 'Night'
    elif pickup_hour < 12:
        time_of_day = 'Morning'
    elif pickup_hour < 18:
        time_of_day = 'Afternoon'
    else:
        time_of_day = 'Evening'

    # Increment count for the payment type at this time of day
    if (time_of_day, row.payment_type) in payment_counts:
        payment_counts[(time_of_day, row.payment_type)] += 1
    else:
        payment_counts[(time_of_day, row.payment_type)] = 1

# Find the most common payment type for each time of day
most_common_payment = {}
for time_of_day in ['Night', 'Morning', 'Afternoon', 'Evening']:
    max_count = 0
    most_common = None
    for key, count in payment_counts.items():
        if key[0] == time_of_day and count > max_count:
            most_common = key[1]
            max_count = count
    most_common_payment[time_of_day] = most_common

# Print the results
print("The most common payment for each time of day is:")
for time_of_day, payment_type in most_common_payment.items():
    print(f"{time_of_day}: {payment_type if payment_type is not None else 'No data available'}")


The most common payment for each time of day is:
Night: 1
Morning: 1
Afternoon: 1
Evening: 1


In [49]:
# g) What is the average tip amount per passenger count?
# query the trip_data table to find the average tip amount per passenger count
# Fetch all rows from Cassandra
rows = session.execute("SELECT passenger_count, tip_amount FROM trip_data")

# Initialize dictionaries to store total tip amount and passenger count for each passenger count
total_tip_amount = {}
passenger_count = {}

# Iterate over rows and update total tip amount and passenger count for each passenger count
for row in rows:
    count = row.passenger_count
    tip = row.tip_amount

    if count in total_tip_amount:
        total_tip_amount[count] += tip
        passenger_count[count] += 1
    else:
        total_tip_amount[count] = tip
        passenger_count[count] = 1

# Calculate average tip amount for each passenger count
average_tip_amount = {}
for count, total in total_tip_amount.items():
    average_tip_amount[count] = total / passenger_count[count]

# Print the result
for count, avg_tip in average_tip_amount.items():
    print(f"Passenger Count: {count}, Average Tip Amount: {avg_tip}")

Passenger Count: 1.0, Average Tip Amount: 1.858077710697463
Passenger Count: 3.0, Average Tip Amount: 1.7751891286801502
Passenger Count: 2.0, Average Tip Amount: 1.83279797850484
Passenger Count: 6.0, Average Tip Amount: 1.9642218252200774
Passenger Count: 5.0, Average Tip Amount: 1.9095819204203826
Passenger Count: 0.0, Average Tip Amount: 1.9354922311293647
Passenger Count: 4.0, Average Tip Amount: 2.0218032819977223


In [52]:
# h) What are the best 5 locations for drivers to pick up passengers from? (based on the trip count)
# query the trip_data table to find the best 5 locations for drivers to pick up passengers from
# Fetch all rows from Cassandra
rows = session.execute("SELECT pickup_location FROM trip_data")

# Initialize dictionary to store pickup counts for each location
pickup_counts = {}

# Count the number of pickups at each location
for row in rows:
    location_id = row.pickup_location
    if location_id in pickup_counts:
        pickup_counts[location_id] += 1
    else:
        pickup_counts[location_id] = 1

# Sort pickup counts in descending order and get the top 5 locations
top_locations = sorted(pickup_counts.items(), key=lambda x: x[1], reverse=True)[:5]

# Print the top 5 pickup locations
for idx, (location_id, count) in enumerate(top_locations, 1):
    print(f"{idx}. Pickup Location: {location_id}, Pickup Count: {count}")

1. Pickup Location: 237, Pickup Count: 806
2. Pickup Location: 161, Pickup Count: 757
3. Pickup Location: 236, Pickup Count: 722
4. Pickup Location: 230, Pickup Count: 705
5. Pickup Location: 162, Pickup Count: 686


In [54]:
# Iterate over the top 5 pickup locations
for x, (zone_id, count) in enumerate(top_locations, 1):
    # Search for the location ID in the zone_data DataFrame
    location_info = zone_data[zone_data['zone_id'] == location_id]

    # Print the information if the location is found
    if not location_info.empty:
        print(f"{x}. Pickup Location: {zone_id}, Pickup Count: {count}")
        print(location_info)
    else:
        print(f"No information found for pickup location with ID: {zone_id}")

1. Pickup Location: 237, Pickup Count: 806
     zone_id              zone_name    borough  \
233      237  Upper East Side South  Manhattan   

                                             zone_geom  
233  POLYGON((-73.9656696 40.7628045559999, -73.965...  
2. Pickup Location: 161, Pickup Count: 757
     zone_id              zone_name    borough  \
233      237  Upper East Side South  Manhattan   

                                             zone_geom  
233  POLYGON((-73.9656696 40.7628045559999, -73.965...  
3. Pickup Location: 236, Pickup Count: 722
     zone_id              zone_name    borough  \
233      237  Upper East Side South  Manhattan   

                                             zone_geom  
233  POLYGON((-73.9656696 40.7628045559999, -73.965...  
4. Pickup Location: 230, Pickup Count: 705
     zone_id              zone_name    borough  \
233      237  Upper East Side South  Manhattan   

                                             zone_geom  
233  POLYGON((-73.9656696